In [ ]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent
from langchain import OpenAI, SerpAPIWrapper

from langchain import SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from rw_formula import RW_Calc

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [ ]:
# from typing import List, Tuple, Any, Union
# from langchain.schema import AgentAction, AgentFinish


# class DBAgent(BaseMultiActionAgent):
#     """DataBase and Calculation Agent."""

#     @property
#     def input_keys(self):
#         return ["input"]

#     def plan(
#         self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
#     ) -> Union[List[AgentAction], AgentFinish]:
#         """Given input, decided what to do.

#         Args:
#             intermediate_steps: Steps the LLM has taken to date,
#                 along with observations
#             **kwargs: User inputs.

#         Returns:
#             Action specifying what tool to use.
#         """
        
#         if len(intermediate_steps) == 0:
#             return [
#                 AgentAction(tool="rwa_db", tool_input=kwargs["input"], log=""),
#             ]
#         elif len(intermediate_steps) == 1:
#             return [
#                 AgentAction(tool="rw_calculator", tool_input=kwargs["input"], log=""),
#             ]
#         else:
#             return AgentFinish()
            

#     async def aplan(
#         self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
#     ) -> Union[List[AgentAction], AgentFinish]:
#         """Given input, decided what to do.

#         Args:
#             intermediate_steps: Steps the LLM has taken to date,
#                 along with observations
#             **kwargs: User inputs.

#         Returns:
#             Action specifying what tool to use.
#         """
#         if len(intermediate_steps) == 0:
#             return [
#                 AgentAction(tool="rwa_db", tool_input=kwargs["input"], log=""),
#             ]
#         elif len(intermediate_steps) == 1:
#             return [
#                 AgentAction(tool="rw_calculator", tool_input=kwargs["input"], log=""),
#             ]
#         else:
#             return AgentFinish()

In [ ]:
agent = DBAgent()

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [ ]:
# agent_executor.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")

#### New Try

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

from rw_formula import RW_Calc

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = ChatOpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="rw_calculator",
        func=RW_Calc,
        description="calculates a loan risk weight"
    ),    
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
agent.run("Report the total RWA and EAD per SEGMENT.")

#### New Try 2

In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory

from rw_formula import RW_Calc2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
llm = OpenAI(temperature=0)
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc2,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),     
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [4]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [5]:
agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    handle_parsing_errors=True
)

In [6]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: rwa_db
Action Input: Read the information for Client 'sfman997ux8036' in RWA_122022 table

> Entering new SQLDatabaseChain chain...
Read the information for Client 'sfman997ux8036' in RWA_122022 table
SQLQuery:SELECT "CLIENT_NUMBER", "ACCOUNT_NUMBER", "PRODUCT", "STATUS", "SEGMENT", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_MORTGAGE", "F_REVOLVING", "F_LARGE_FIN", "RW", "RWA" FROM "RWA_122022" WHERE "CLIENT_NUMBER" = 'sfman997ux8036' LIMIT 5;
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:The information for Client 'sfman997ux8036' in RWA_122022 table is: CLIENT_NUMBER: sfman997ux8036, ACCOUNT_NUMBER: 1324710203qiph20, PRODUCT: CREDIT_CARD, STATUS: Performi

'The risk weight for the given parameters is 0.3950618648985617.'

In [57]:
Segment_s = 'Segment: Retail'

In [58]:
Segment = (Segment_s.split(":")[1].strip()).replace("'", "")

In [59]:
Segment

'Retail'

In [ ]:
# Action Input: Segment: Retail, PD: 0.0493036, LGD: 0.3092, m: 1.15616438, Large_Fin: N, size: 42.58029755, mortgage: N, revolving: Y

In [65]:
# Action Input: Segment = 'Retail', PD = 0.0246518 * 2, LGD = 0.3092, m = 1.15616438, Large_Fin = 'N', size = 42.58029755, mortgage = 'N', revolving = 'Y'

In [75]:
a = ("Segment: Retail, PD: 0.0493036, LGD: 0.3092, m: 1.15616438, Large_Fin: N, size: 42.58029755, mortgage: N, revolving: Y").split(",")

# Segment = (Segment_s.split("=")[1].strip()).replace("'", "")
# PD = float(eval(PD_s.split("=")[1].strip()))
# LGD = float(eval(LGD_s.split("=")[1].strip()))
# m = float(eval(m_s.split("=")[1].strip()))
# Large_Fin = (Large_Fin_s.split("=")[1].strip()).replace("'", "")
# size = float(eval(size_s.split("=")[1].strip()))
# mortgage = (mortgage_s.split("=")[1].strip()).replace("'", "")
# revolving = (revolving_s.split("=")[1].strip()).replace("'", "")

In [76]:
a

['Segment: Retail',
 ' PD: 0.0493036',
 ' LGD: 0.3092',
 ' m: 1.15616438',
 ' Large_Fin: N',
 ' size: 42.58029755',
 ' mortgage: N',
 ' revolving: Y']

In [85]:
float(eval(a[1].split(":")[1].strip().replace("'", "")))

0.0493036

In [ ]:
agent.run("What is the sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

In [62]:
agent.run("Report the all information for Client 'sfman997ux8036' in RWA_122022.")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: rwa_db
Action Input: Report the all information for Client 'sfman997ux8036' in RWA_122022.

> Entering new SQLDatabaseChain chain...
Report the all information for Client 'sfman997ux8036' in RWA_122022.
SQLQuery:SELECT * FROM "RWA_122022" WHERE "CLIENT_NUMBER" = 'sfman997ux8036';
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:The information for Client 'sfman997ux8036' in RWA_122022 is: CLIENT_NUMBER: sfman997ux8036, ACCOUNT_NUMBER: 1324710203qiph20, PRODUCT: CREDIT_CARD, STATUS: Performing, SEGMENT: Retail, EXPOSURE_DRAWN: 0.0, EXPOSURE_UNDRAWN: 18728.13876, EXPOSURE: 18728.13876, EAD: 11004.65713, EAD_COLL: 0.0, PD: 0.0246518, LGD: 0.3092, SIZE: 42.58029755, MATURITY: 1.15616438, F_MORTGAGE: N, F_REVOLVING: Y, F_LARG

'Is there anything else I can help you with?'

In [63]:
agent.run("Calculate and report the RW of Client 'sfman997ux8036' if the PD doubles.")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: risk weight calculator
Action Input: Segment: Bank, PD: double, LGD: 0.5, m: 5, Large_Fin: N, size: 10, mortgage: N, revolving: N

TypeError: RW_Calc2() missing 7 required positional arguments: 'PD', 'LGD', 'm', 'Large_Fin', 'size', 'mortgage', and 'revolving'